# 本文件说明
- 读取数据、预处理数据

# 基本设置

In [1]:
import xlrd

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor_circ

from toolkits.setup.specific_func import contain_ch, set_ch_pd
set_ch_pd()
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.977 seconds.
Prefix dict has been built succesfully.


In [2]:
%load_ext autoreload
%autoreload 2

## 一些函数

In [3]:
def predict_right(folder, file_list):
    '''预测正确数据'''
    right_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Right']
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
#             tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
#             print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            right_data = pd.concat([right_data, tmp_data], axis = 0)    
    return right_data

In [4]:
def correct_wrong_data(folder, file_list):
    '''预测错误修正后数据'''
    correct_wrong_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
    #         tmp_data = tmp_data[tmp_data['R_W'] == 'Wrong']
            print('去删除前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data[tmp_data['备注'] != '删除']
    #         tmp_data = tmp_data.dropna(subset = ['raw_title'], axis = 0)
            print('去删除后： ', tmp_data.shape, file_name, sheet_name)

            tmp_data['人工判断'] = tmp_data.apply(lambda x:1 if x['label'] == x['备注'] else 0, axis = 1)
            tmp_data['算法判断'] = tmp_data.apply(lambda x:1 if x['predict_label'] == x['备注'] else 0, axis = 1)
            print('人工误判率： %0.2f'%(1 - tmp_data['人工判断'].sum() / tmp_data.shape[0]),
                  '算法误判率： %0.2f'%(1 - tmp_data['算法判断'].sum() / tmp_data.shape[0]))
            print()

            correct_wrong_data = pd.concat([correct_wrong_data, tmp_data], axis = 0)
    return correct_wrong_data

In [5]:
def get_new_data(folder, file_list):
    '''新补充数据'''
    new_data = pd.DataFrame()
    for file_name in file_list:
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:    
            tmp_data = pd.read_excel(currentPath, sheet_name)
            print(tmp_data.columns)
            tmp_data = tmp_data[['label', 'title', 'content']]
            print('去空值前： ', tmp_data.shape, file_name, sheet_name)
            tmp_data = tmp_data.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data.shape, file_name, sheet_name)
            new_data = pd.concat([new_data, tmp_data], axis = 0)    
    return new_data

In [6]:
def get_PR_plot_data(feedback_data, date, plot_data):
    y_pred_class = feedback_data[feedback_data['date'] == date]['predict_label'].tolist()
    y_test = feedback_data[feedback_data['date'] == date]['label'].tolist()
    print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
    print(metrics.classification_report(y_test, y_pred_class))
    print('confusion_matrix: ')
    print( metrics.confusion_matrix(y_test, y_pred_class))
    print()       
    
    classification_report = metrics.classification_report(y_test, y_pred_class)
    lines = classification_report.split('\n')
    # plot_data = []
    for index, line in enumerate(lines[2 : len(lines)]):
        t = line.strip().split()
        if len(t) < 2: continue    
        if (index + 2) < (len(lines) - 2):
            t = [float(v) if i > 0 else v for i, v in enumerate(t)]
            plot_data.append(t + [date])
        else :
            t = [float(v) if i > 2 else v for i, v in enumerate(t) ]
            plot_data.append(['avg/total'] + t[3:] + [date])    
            
    return plot_data

In [7]:
def get_feedback_data(folder, file_list):
    '''模型预测后反馈数据'''
    feedback_data = pd.DataFrame()
    label_list = []
    sum_list = []
    right_list = []
    plot_data = []
    for file_name in file_list:
        print('  ----    ', file_name)
        date = file_name.split('_')[0]
        currentPath  = '{0}/{1}'.format(folder, file_name)
        excel = xlrd.open_workbook(currentPath)
        sheet_names = [sheet.name for sheet in excel.sheets()]
        for sheet_name in sheet_names:   
            tmp_data = pd.read_excel(currentPath, sheet_name)    
            tmp_data = tmp_data[tmp_data['label'] != '删除']
            tmp_data_1 = tmp_data[tmp_data['label'] == sheet_name]

            label_list.append(sheet_name)
            sum_list.append(tmp_data.shape[0])
            right_list.append(tmp_data_1.shape[0])
            print('类别：', sheet_name, '总数：', tmp_data.shape[0], 
                  '正确数', tmp_data_1.shape[0], 
                  '正确比例', tmp_data_1.shape[0]/tmp_data.shape[0])
            
            if 'predict_label' in tmp_data_1.columns:
                tmp_data_1 = tmp_data[['predict_label', 'label', 'title', 'content']]
            else :
                tmp_data_1 = tmp_data[['label', 'title', 'content']]
                if contain_ch(sheet_name) :
                    tmp_data_1['predict_label'] = sheet_name
                else :
                    tmp_data_1['predict_label'] = ''
                
            tmp_data_1['date'] = date
            print('去空值前： ', tmp_data_1.shape, file_name, sheet_name)
            tmp_data_1 = tmp_data_1.dropna(subset = ['label'], axis = 0)
            print('去空值后： ', tmp_data_1.shape, file_name, sheet_name)
            feedback_data = pd.concat([feedback_data, tmp_data_1], axis = 0)
            print() 
                        
        plot_data = get_PR_plot_data(feedback_data, date, plot_data) 
            
    PR_data = feedback_data[['date', 'predict_label', 'label']]
    feedback_data = feedback_data.drop('date', axis = 1)
    feedback_data = feedback_data.drop('predict_label', axis = 1)
    PR_plot_data  = pd.DataFrame(plot_data, columns = ['label', 'precision', 'recall', 
                                                    'f1-score', 'support', 'date'])     
    
    return feedback_data, PR_data, PR_plot_data

In [8]:
def pre_save(pre_func, data, save_folder):
    print(data.shape)
    print('save_folder: ', save_folder)
    
    titles = pre_func(data['title'].tolist())
    print('title num: ', len(titles))
    save_filename = save_folder + 'corpus/titles.txt'
    fid = open(save_filename, "w+", encoding='UTF-8')
    for line in titles:
        fid.write(line + '\n')
    fid.close()  
    
#     print(len(data['content'].tolist()))
    contents = pre_func(data['content'].tolist())
    print('content num: ', len(contents))
    print(contents[0])
    # contents = [re.sub(r'[a-z]*', '', x) for x in contents]
    # print(len(contents))
    # print(contents[:2])
    coprus_save_filename = save_folder + 'corpus/contents.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in contents:
        f.write(line + '\n')
    f.close()    

    label = data['label'].tolist()
    print('label num: ', len(label))
    coprus_save_filename = save_folder + 'corpus/labels.txt'
    f = open(coprus_save_filename, "w+", encoding='UTF-8')
    for line in label:
        f.write(str(line) + '\n')
    f.close()

    data.to_excel(save_folder + 'corpus/title_content_label.xlsx', index = False)
    

# 西藏教育厅：1 八分类

## 读取数据：0 不相关 1 相关

In [9]:
folder = 'cor_data/20181120'

### 预测正确数据

In [10]:
# file_list_1 = ['circ_class_sel_else_20180827.xlsx',]
# circ_raw_class_1 = predict_right(folder, file_list_1)

# circ_raw_class_1['label'] = circ_raw_class_1['label'].apply(lambda x:x.strip())    
# circ_raw_class_1 = circ_raw_class_1[['label', 'raw_title', 'raw_content']]
# circ_raw_class_1.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_1.shape)
# circ_raw_class_1.head()

In [11]:
# circ_raw_class_1['label'].value_counts()

### 预测错误修正后数据

In [12]:
# file_list_2 = ['circ_class_sel_20180827-1.xlsx','circ_class_sel_20180827-2.xlsx']

# circ_raw_class_2 = correct_wrong_data(folder, file_list_2)
# circ_raw_class_2['label'] = circ_raw_class_2['label'].apply(lambda x:x.strip())  
# circ_raw_class_2 = circ_raw_class_2[['备注', 'raw_title', 'raw_content']]
# circ_raw_class_2.columns = ['label', 'title', 'content']
# print('总数：', circ_raw_class_2.shape)
# circ_raw_class_2.head()

In [13]:
# circ_raw_class_2['label'].value_counts()

### 新补充数据

In [14]:
file_list_3 = ['181120西藏教育厅（与教育相关否）.xlsx',
               ] # 多个 sheet

circ_raw_class_3 = get_new_data(folder, file_list_3)

# circ_raw_class_3['label'] = circ_raw_class_3['label'].apply(lambda x:x.strip())    
print('总数：', circ_raw_class_3.shape)
circ_raw_class_3.head()

Index(['title', 'content', 'label'], dtype='object')
去空值前：  (616, 3) 181120西藏教育厅（与教育相关否）.xlsx 不相关
去空值后：  (616, 3) 181120西藏教育厅（与教育相关否）.xlsx 不相关
Index(['title', 'content', 'label'], dtype='object')
去空值前：  (433, 3) 181120西藏教育厅（与教育相关否）.xlsx 相关
去空值后：  (433, 3) 181120西藏教育厅（与教育相关否）.xlsx 相关
总数： (1049, 3)


,label,title,content
0,1,萨基会“回到拉萨”——足球时间圆梦世界屋脊,"正是他们的足球梦想让萨马兰奇足球时间三星足球教室不远万里克服重重困难来到这里助他们圆梦足球,西藏优秀中学生放眼祖国公益活动 本次萨马兰奇足球时间三星足球教室也是在5100西藏冰川矿泉水有限公司的鼎力协助下进驻拉萨,训后教练团队表示小球员们的身体素质和个人技能在同龄的孩子中已经是相当的优秀了,萨马兰奇足球时间三星足球教室走进拉萨当雄县中学,萨马兰奇足球时间才得以走进拉萨,教练员为大家做训前动员 接受训练的小球员们普遍有着良好的身体素质与个人技巧,热爱足球的80名青少年,希望孩子们在足球的道路上越走越远,圆梦足球,成立了5100西藏体育教育专项基金"
1,1,我，和梁思成、刘长春在东大的日子,"东北大学优秀社会实践团队,均获得东北大学优秀学生称号和校优秀学生一等奖学金,连续两年获得东北大学优秀学生标兵、国家奖学金及东北大学优秀学生一等奖学金,东北大学优秀团队,东北大学优秀学生一等奖学金,还积极参加社会实践活动,团队成员在学习、工作活动等其他各个方面都取得了优异的成绩,东北大学优秀学生,并获得社会实践优秀团队和优秀个人,荣获东北大学工商管理学院优秀党员等称号"
2,1,六十五载桃李路 千言万语华职情~~~,"由泉州华职语文学科组主办的第九届普通话形象大使比赛暨,比赛的选手则是来自2018级各个专业的同学,就在明天 咱一起华职校园回忆青春 文字,华职学子展现出独一无二的青春风采, 此次比赛也进一步推广了普通话在校园的使用,演讲比赛中展现出积极向上的精神面貌,评委老师们主要从演讲内容、语音、情感、流畅度、仪态等方面为选手进行打分,演讲比赛隆重举行,有的选手一展歌喉, 幼教、财会专业的选手围绕着"
3,0,我市90后医生莫就如援藏归来,"莫就如与同事分享援藏经历时间,莫就如也会向市妇幼保健院的专家请教,还会派驻像莫就如一样的优秀医生到西藏去,在莫就如的努力下,把在阳江掌握的技术带到雪域高原,为了藏区孩子的健康,更好地施救患儿,是阳江市妇幼保健院新生儿科一名医生,莫就如到西藏在林芝市人民医院儿科报到,要用自己的医疗技术给孩子带来健康"
4,0,关于藏医传承，堆龙德庆区这样做,"拉萨市堆龙德庆区卫计委委托西藏母子保健协会举办的拉萨市堆龙德庆区乡村医生藏医药技术骨干培训班于11月14日正式开班,在拉萨市堆龙德庆区乡村医生藏医药技术骨干培训开班典礼上,此次培训的目标旨在提升基层藏医医疗机构的藏医传统诊疗服务,这一期培训是协会与堆龙德庆区卫计委第二次合作项目,开班典礼 拉萨市堆龙德庆区卫计委医政科夏辉老师、藏医科德吉康珠老师,感谢堆龙德庆区领导再一次把培训任务交给协会,自治区母子保健协会副会长丹增曲珍等相关工作人员以及来自堆龙德庆区的16名乡村医生参加了开班典礼,培训内容按照拉萨市堆龙德庆区卫计委要求,按照《基层藏医药服务能力提升工程项目任务方案》,藏医培训 藏医药是中国医学宝库中一颗璀璨的明珠"


In [15]:
circ_raw_class_3['label'].value_counts()

0    625
1    424
Name: label, dtype: int64

### 模型预测后反馈数据

In [17]:
# file_list_4 = ['20180904_circ_class_feedback.xlsx', '20180905_circ_class_feedback.xlsx',
#                '20180912_circ_class_feedback.xlsx','20180920_circ_class_feedback.xlsx',
#                '20180925_circ_class_feedback.xlsx', '20181008_circ_class_feedback.xlsx',
#                '20181019_circ_class_feedback.xlsx', '20181112_circ_class_feedback.xlsx'
#               ] # 多个 sheet

# circ_raw_class_4, PR_data, PR_plot_data = get_feedback_data(folder, file_list_4)

# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [18]:
# circ_raw_class_4['label'].value_counts()

#### PR

In [19]:
# y_pred_class = PR_data['predict_label'].tolist()
# y_test = PR_data['label'].tolist()
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))
# # PR_data.head()

In [20]:
# PR_plot_data
# PR_plot_data = PR_plot_data.reset_index('date')

In [21]:
# PR_plot_data[PR_plot_data['label'] == 'avg/total'][[ 'date', 'precision', 'recall' ,'f1-score']].plot(x = 'date')
# PR_plot_data[PR_plot_data['label'] == 'avg/total']

In [22]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'precision']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

In [23]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'recall']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

In [24]:
# P_data = PR_plot_data[PR_plot_data['label'] != 'avg/total'][ ['date', 'label', 'f1-score']]
# P_data.set_index(['date','label']).unstack('label').plot(figsize = (15,6))
# P_data.set_index(['date','label']).unstack('label')

### 补录与预警数据

In [25]:
# file_list_5 = ['20180917_circ_class_feedback.xlsx'
#               ] 
# add_warn_data = pd.DataFrame()
# for file_name in file_list_5:
#     currentPath  = '{0}/{1}'.format(folder, file_name)
#     excel = xlrd.open_workbook(currentPath)
#     sheet_names = [sheet.name for sheet in excel.sheets()]
#     for sheet_name in sheet_names:   
#         tmp_data = pd.read_excel(currentPath, sheet_name)    
#         tmp_data = tmp_data[tmp_data['label'] != '删除']
#         add_warn_data = pd.concat([add_warn_data, tmp_data], axis = 0)

# circ_raw_class_5 = add_warn_data[['label', 'title', 'content']]

In [26]:
# from sklearn import metrics

# y_pred_class = add_warn_data['predict_label']
# y_test = add_warn_data['label']
# print('accuracy_score: ', metrics.accuracy_score(y_test, y_pred_class)) # 指所有分类正确的百分比
# print(metrics.classification_report(y_test, y_pred_class))
# print('confusion_matrix: ')
# print( metrics.confusion_matrix(y_test, y_pred_class))

In [27]:
# circ_raw_class_5['label'].value_counts()

## 合并数据

In [28]:
# file_list_4 = ['20181019_circ_class_feedback.xlsx'] # 多个 sheet
# circ_raw_class_4, PR_data, PR_plot_data = get_feedback_data(folder, file_list_4)
# circ_raw_class_4['label'] = circ_raw_class_4['label'].apply(lambda x:x.strip())    
# print('总数：', circ_raw_class_4.shape)
# circ_raw_class_4.head()

In [17]:
# circ_raw_class = pd.concat([circ_raw_class_1, circ_raw_class_2,
#                           circ_raw_class_3, circ_raw_class_4,
#                            circ_raw_class_5], 
#                          axis = 0)

circ_raw_class = circ_raw_class_3
circ_raw_class['title_content'] = circ_raw_class['title'].astype(str) + '。' + circ_raw_class['content'].astype(str)
# circ_raw_class['label'] = circ_raw_class['label'].apply(lambda x:x.strip())
circ_raw_class['index'] = range(circ_raw_class.shape[0])
print('去重前： ', circ_raw_class.shape)
circ_raw_class = circ_raw_class.drop_duplicates(subset = 'title')
print('去重后： ', circ_raw_class.shape)
circ_raw_class = circ_raw_class.drop_duplicates(subset = 'content')
print('去重后： ', circ_raw_class.shape)
circ_raw_class.head()

去重前：  (1049, 5)
去重后：  (958, 5)
去重后：  (933, 5)


,label,title,content,title_content,index
0,1,萨基会“回到拉萨”——足球时间圆梦世界屋脊,"正是他们的足球梦想让萨马兰奇足球时间三星足球教室不远万里克服重重困难来到这里助他们圆梦足球,西藏优秀中学生放眼祖国公益活动 本次萨马兰奇足球时间三星足球教室也是在5100西藏冰川矿泉水有限公司的鼎力协助下进驻拉萨,训后教练团队表示小球员们的身体素质和个人技能在同龄的孩子中已经是相当的优秀了,萨马兰奇足球时间三星足球教室走进拉萨当雄县中学,萨马兰奇足球时间才得以走进拉萨,教练员为大家做训前动员 接受训练的小球员们普遍有着良好的身体素质与个人技巧,热爱足球的80名青少年,希望孩子们在足球的道路上越走越远,圆梦足球,成立了5100西藏体育教育专项基金","萨基会“回到拉萨”——足球时间圆梦世界屋脊。正是他们的足球梦想让萨马兰奇足球时间三星足球教室不远万里克服重重困难来到这里助他们圆梦足球,西藏优秀中学生放眼祖国公益活动 本次萨马兰奇足球时间三星足球教室也是在5100西藏冰川矿泉水有限公司的鼎力协助下进驻拉萨,训后教练团队表示小球员们的身体素质和个人技能在同龄的孩子中已经是相当的优秀了,萨马兰奇足球时间三星足球教室走进拉萨当雄县中学,萨马兰奇足球时间才得以走进拉萨,教练员为大家做训前动员 接受训练的小球员们普遍有着良好的身体素质与个人技巧,热爱足球的80名青少年,希望孩子们在足球的道路上越走越远,圆梦足球,成立了5100西藏体育教育专项基金",0
1,1,我，和梁思成、刘长春在东大的日子,"东北大学优秀社会实践团队,均获得东北大学优秀学生称号和校优秀学生一等奖学金,连续两年获得东北大学优秀学生标兵、国家奖学金及东北大学优秀学生一等奖学金,东北大学优秀团队,东北大学优秀学生一等奖学金,还积极参加社会实践活动,团队成员在学习、工作活动等其他各个方面都取得了优异的成绩,东北大学优秀学生,并获得社会实践优秀团队和优秀个人,荣获东北大学工商管理学院优秀党员等称号","我，和梁思成、刘长春在东大的日子。东北大学优秀社会实践团队,均获得东北大学优秀学生称号和校优秀学生一等奖学金,连续两年获得东北大学优秀学生标兵、国家奖学金及东北大学优秀学生一等奖学金,东北大学优秀团队,东北大学优秀学生一等奖学金,还积极参加社会实践活动,团队成员在学习、工作活动等其他各个方面都取得了优异的成绩,东北大学优秀学生,并获得社会实践优秀团队和优秀个人,荣获东北大学工商管理学院优秀党员等称号",1
2,1,六十五载桃李路 千言万语华职情~~~,"由泉州华职语文学科组主办的第九届普通话形象大使比赛暨,比赛的选手则是来自2018级各个专业的同学,就在明天 咱一起华职校园回忆青春 文字,华职学子展现出独一无二的青春风采, 此次比赛也进一步推广了普通话在校园的使用,演讲比赛中展现出积极向上的精神面貌,评委老师们主要从演讲内容、语音、情感、流畅度、仪态等方面为选手进行打分,演讲比赛隆重举行,有的选手一展歌喉, 幼教、财会专业的选手围绕着","六十五载桃李路 千言万语华职情~~~。由泉州华职语文学科组主办的第九届普通话形象大使比赛暨,比赛的选手则是来自2018级各个专业的同学,就在明天 咱一起华职校园回忆青春 文字,华职学子展现出独一无二的青春风采, 此次比赛也进一步推广了普通话在校园的使用,演讲比赛中展现出积极向上的精神面貌,评委老师们主要从演讲内容、语音、情感、流畅度、仪态等方面为选手进行打分,演讲比赛隆重举行,有的选手一展歌喉, 幼教、财会专业的选手围绕着",2
3,0,我市90后医生莫就如援藏归来,"莫就如与同事分享援藏经历时间,莫就如也会向市妇幼保健院的专家请教,还会派驻像莫就如一样的优秀医生到西藏去,在莫就如的努力下,把在阳江掌握的技术带到雪域高原,为了藏区孩子的健康,更好地施救患儿,是阳江市妇幼保健院新生儿科一名医生,莫就如到西藏在林芝市人民医院儿科报到,要用自己的医疗技术给孩子带来健康","我市90后医生莫就如援藏归来。莫就如与同事分享援藏经历时间,莫就如也会向市妇幼保健院的专家请教,还会派驻像莫就如一样的优秀医生到西藏去,在莫就如的努力下,把在阳江掌握的技术带到雪域高原,为了藏区孩子的健康,更好地施救患儿,是阳江市妇幼保健院新生儿科一名医生,莫就如到西藏在林芝市人民医院儿科报到,要用自己的医疗技术给孩子带来健康",3
4,0,关于藏医传承，堆龙德庆区这样做,"拉萨市堆龙德庆区卫计委委托西藏母子保健协会举办的拉萨市堆龙德庆区乡村医生藏医药技术骨干培训班于11月14日正式开班,在拉萨市堆龙德庆区乡村医生藏医药技术骨干培训开班典礼上,此次培训的目标旨在提升基层藏医医疗机构的藏医传统诊疗服务,这一期培训是协会与堆龙德庆区卫计委第二次合作项目,开班典礼 拉萨市堆龙德庆区卫计委医政科夏辉老师、藏医科德吉康珠老师,感谢堆龙德庆区领导再一次把培训任务交给协会,自治区母子保健协会副会长丹增曲珍等相关工作人员以及来自堆龙德庆区的16名乡村医生参加了开班典礼,培训内容按照拉萨市堆龙德庆区卫计委要求,按照《基层藏医药服务能力提升工程项目任务方案》,藏医培训 藏医药是中国医学宝库中一颗璀璨的明珠","关于藏医传承，堆龙德庆区这样做。拉萨市堆龙德庆区卫计委委托西藏母子保健协会举办的拉萨市堆龙德庆区乡村医生藏医药技术骨干培训班于11月14日正式开班,在拉萨市堆龙德庆区乡村医生藏医药技术骨干培训开班典礼上,此次培训的目标旨在提升基层藏医医疗机构的藏医传统诊疗服务,这一期培训是协会与堆龙德庆区卫计委第二次合作项目,开班典礼 拉萨市堆龙德庆区卫计委医政科夏辉老师、藏医科德吉康珠老师,感谢堆龙德庆区领导再一次把培训任务交给协会,自治区母子保健协会副会长丹增曲珍等相关工作人员以及来自堆龙德庆区的16名乡村医生参加了开班典礼,培训内容按照拉萨市堆龙德庆区卫计委要求,按照《基层藏医药服务能力提升工程项目任务方案》,藏医培训 藏医药是中国医学宝库中一颗璀璨的明珠",4


In [18]:
print(circ_raw_class.shape)
circ_raw_class.groupby('label')['title'].count()

(933, 5)


label
0    605
1    328
Name: title, dtype: int64

## 预处理

In [19]:
save_folder = 'cor_result/'
pre_save(pre_cor_circ.handle_contents, circ_raw_class, save_folder)

(933, 5)
save_folder:  cor_result/
title num:  933
content num:  933
足球 梦想 萨马兰奇 足球 时间 三星 足球 教室 不远万里 克服 重重困难 来到 助 圆梦 足球 西藏 优秀 中学生 放眼 祖国 公益活动 本次 萨马兰奇 足球 时间 三星 足球 教室 西藏 冰川 矿泉水 有限公司 鼎力 协助 进驻 拉萨 训后 教练 团队 小球员 身体素质 技能 孩子 中 优秀 萨马兰奇 足球 时间 三星 足球 教室 走进 拉萨 当雄县 中学 萨马兰奇 足球 时间 得以 走进 拉萨 教练员 做 训前 动员 接受 训练 小球员 身体素质 技巧 热爱 足球 名 青少年 希望 孩子 足球 道路 越走越远 圆梦 足球 成立 西藏 体育 教育 专项基金 
label num:  933
